In [ ]:
import sys
import os
sys.path.append("./../bin/Release")
sys.path.append("./../")
import numpy as np
import time
from scipy.optimize import differential_evolution, minimize
import matplotlib.pyplot as plt
import pandas as pd
from pyminion.test import MWUT

In [ ]:
goptimum_cec22 = {
    1 : 300, 2: 400, 3: 600, 4: 800, 5:900, 6:1800, 7:2000, 8:2200, 9:2300, 10: 2400, 11:2600, 12: 2700
}
goptimum_cec20 = {
    1 : 100, 2: 1100, 3: 700, 4: 1900, 5:1700, 6:1600, 7:2100, 8:2200, 9:2400, 10: 2500
}
goptimum_cec17 = { i:100*i for i in range(1, 31)}
goptimum_cec20 = np.array(list(goptimum_cec20.values()))
goptimum_cec22 = np.array(list(goptimum_cec22.values()))
goptimum_cec17 = np.array(list(goptimum_cec17.values()))

resFolder = "./../bin/"

# CEC2014

In [ ]:
filename = resFolder+"results_2014_NLSHADE_RSP_10_100000.txt"
data = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2014_ARRDE_10_100000.txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2014_j2020_10_100000.txt"
data2 = np.loadtxt(filename, delimiter='\t')
#filename = resFolder+"results_2014_LSHADE_10_100000.txt"
#data3 = np.loadtxt(filename, delimiter='\t')
#filename = resFolder+"results_2014_jSO_10_100000.txt"
#data4 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2014_LSRTDE_10_100000.txt"
data5 = np.loadtxt(filename, delimiter='\t')
#filename = resFolder+"results_2017_JADE_10_100000.txt"
#data6 = np.loadtxt(filename, delimiter='\t')
"""
df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec17), axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec17), axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec17), axis=0), 
    #"LSHADE" : np.mean(np.abs(data3-goptimum_cec17), axis=0), 
    #"jSO" : np.mean(np.abs(data4-goptimum_cec17), axis=0), 
    #"LSRTDE" : np.mean(np.abs(data5-goptimum_cec17), axis=0), 
    #"JADE" : np.mean(np.abs(data6-goptimum_cec17), axis=0)
    })
"""

df = pd.DataFrame({
    "NLSHADE_RSP" : [MWUT(data1[:, i], data[:,  i]) for i in range(30)], 
    "j2020" : [MWUT(data1[:, i], data2[:,  i]) for i in range(30)], 
    #"LSHADE" : [MWUT(data1[:, i], data3[:,  i]) for i in range(30)], 
    "LSRTDE" : [MWUT(data1[:, i], data5[:,  i]) for i in range(30)], 
    })
for c in df.columns : print(c, np.mean(df[c]))
#display(df)


In [ ]:
filename = resFolder+"results_2014_NLSHADE_RSP_30_300000.txt"
data = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2014_ARRDE_30_300000.txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2014_j2020_30_300000.txt"
data2 = np.loadtxt(filename, delimiter='\t')
#filename = resFolder+"results_2014_LSHADE_10_100000.txt"
#data3 = np.loadtxt(filename, delimiter='\t')
#filename = resFolder+"results_2014_jSO_10_100000.txt"
#data4 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2014_LSRTDE_30_300000.txt"
data5 = np.loadtxt(filename, delimiter='\t')
#filename = resFolder+"results_2017_JADE_10_100000.txt"
#data6 = np.loadtxt(filename, delimiter='\t')
"""
df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec17), axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec17), axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec17), axis=0), 
    #"LSHADE" : np.mean(np.abs(data3-goptimum_cec17), axis=0), 
    #"jSO" : np.mean(np.abs(data4-goptimum_cec17), axis=0), 
    #"LSRTDE" : np.mean(np.abs(data5-goptimum_cec17), axis=0), 
    #"JADE" : np.mean(np.abs(data6-goptimum_cec17), axis=0)
    })
"""

df = pd.DataFrame({
    "NLSHADE_RSP" : [MWUT(data1[:, i], data[:,  i]) for i in range(30)], 
    "j2020" : [MWUT(data1[:, i], data2[:,  i]) for i in range(30)], 
    #"LSHADE" : [MWUT(data1[:, i], data3[:,  i]) for i in range(30)], 
    "LSRTDE" : [MWUT(data1[:, i], data5[:,  i]) for i in range(30)], 
    })
for c in df.columns : print(c, np.mean(df[c]))
display(df)


# CEC 2017

In [ ]:
filename = resFolder+"results_2017_NLSHADE_RSP_10_100000.txt"
data = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_ARRDE_10_100000.txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_j2020_10_100000.txt"
data2 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_LSHADE_10_100000.txt"
data3 = np.loadtxt(filename, delimiter='\t')
#filename = resFolder+"results_2017_jSO_10_100000.txt"
#data4 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_LSRTDE_10_100000.txt"
data5 = np.loadtxt(filename, delimiter='\t')
#filename = resFolder+"results_2017_JADE_10_100000.txt"
#data6 = np.loadtxt(filename, delimiter='\t')

df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec17)/goptimum_cec17, axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec17)/goptimum_cec17, axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec17)/goptimum_cec17, axis=0), 
    "LSHADE" : np.mean(np.abs(data3-goptimum_cec17)/goptimum_cec17, axis=0), 
    #"jSO" : np.mean(np.abs(data4-goptimum_cec17), axis=0), 
    "LSRTDE" : np.mean(np.abs(data5-goptimum_cec17)//goptimum_cec17, axis=0), 
    #"JADE" : np.mean(np.abs(data6-goptimum_cec17), axis=0)
    })


display(df)
for c in df.columns : print(c, np.mean(df[c]))



print("------------")
df = pd.DataFrame({
    "NLSHADE_RSP" : [MWUT(data1[:, i], data[:,  i]) for i in range(30)], 
    "j2020" : [MWUT(data1[:, i], data2[:,  i]) for i in range(30)], 
    "LSHADE" : [MWUT(data1[:, i], data3[:,  i]) for i in range(30)], 
    "LSRTDE" : [MWUT(data1[:, i], data5[:,  i]) for i in range(30)], 
    })
for c in df.columns : print(c, np.mean(df[c]))
#display(df)


In [ ]:
filename = resFolder+"results_2017_NLSHADE_RSP_30_300000.txt"
data = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_ARRDE_30_300000.txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_j2020_30_300000.txt"
data2 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_LSHADE_30_300000.txt"
data3 = np.loadtxt(filename, delimiter='\t')
#filename = resFolder+"results_2017_jSO_30_300000.txt"
#data4 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_LSRTDE_30_300000.txt"
data5 = np.loadtxt(filename, delimiter='\t')

df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec17)/goptimum_cec17, axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec17)/goptimum_cec17, axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec17)/goptimum_cec17, axis=0), 
    "LSHADE" : np.mean(np.abs(data3-goptimum_cec17)/goptimum_cec17, axis=0), 
    #"jSO" : np.mean(np.abs(data4-goptimum_cec17), axis=0), 
    "LSRTDE" : np.mean(np.abs(data5-goptimum_cec17)//goptimum_cec17, axis=0), 
    #"JADE" : np.mean(np.abs(data6-goptimum_cec17), axis=0)
    })


display(df)
for c in df.columns : print(c, np.mean(df[c]))

print("------------")
df = pd.DataFrame({
    "NLSHADE_RSP" : [MWUT(data1[:, i], data[:,  i]) for i in range(30)], 
    "j2020" : [MWUT(data1[:, i], data2[:,  i]) for i in range(30)], 
    "LSHADE" : [MWUT(data1[:, i], data3[:,  i]) for i in range(30)], 
    "LSRTDE" : [MWUT(data1[:, i], data5[:,  i]) for i in range(30)], 
    })
for c in df.columns : print(c, np.mean(df[c]))

In [ ]:
filename = resFolder+"results_2017_NLSHADE_RSP_50_500000.txt"
data = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_ARRDE_50_500000.txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_j2020_50_500000.txt"
data2 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_LSHADE_50_500000.txt"
data3 = np.loadtxt(filename, delimiter='\t')
#filename = resFolder+"results_2017_jSO_50_500000.txt"
#data4 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_LSRTDE_50_500000.txt"
data5 = np.loadtxt(filename, delimiter='\t')

df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec17)/goptimum_cec17, axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec17)/goptimum_cec17, axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec17)/goptimum_cec17, axis=0), 
    "LSHADE" : np.mean(np.abs(data3-goptimum_cec17)/goptimum_cec17, axis=0), 
    #"jSO" : np.mean(np.abs(data4-goptimum_cec17), axis=0), 
    "LSRTDE" : np.mean(np.abs(data5-goptimum_cec17)//goptimum_cec17, axis=0), 
    #"JADE" : np.mean(np.abs(data6-goptimum_cec17), axis=0)
    })


display(df)
for c in df.columns : print(c, np.mean(df[c]))

print("------------")

df = pd.DataFrame({
    "NLSHADE_RSP" : [MWUT(data1[:, i], data[:,  i]) for i in range(30)], 
    "j2020" : [MWUT(data1[:, i], data2[:,  i]) for i in range(30)], 
    "LSHADE" : [MWUT(data1[:, i], data3[:,  i]) for i in range(30)], 
    "LSRTDE" : [MWUT(data1[:, i], data5[:,  i]) for i in range(30)], 
    })
for c in df.columns : print(c, np.mean(df[c]))

In [ ]:
filename = resFolder+"results_2017_NLSHADE_RSP_100_1000000.txt"
data = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_ARRDE_100_1000000.txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_j2020_100_1000000.txt"
data2 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_LSHADE_100_1000000.txt"
data3 = np.loadtxt(filename, delimiter='\t')
#filename = resFolder+"results_2017_jSO_100_1000000.txt"
#data4 = np.loadtxt(filename, delimiter='\t')
#filename = resFolder+"results_2017_LSRTDE_100_100000.txt"
#data5 = np.loadtxt(filename, delimiter='\t')

df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec17)/goptimum_cec17, axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec17)/goptimum_cec17, axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec17)/goptimum_cec17, axis=0), 
    "LSHADE" : np.mean(np.abs(data3-goptimum_cec17)/goptimum_cec17, axis=0), 
    #"jSO" : np.mean(np.abs(data4-goptimum_cec17), axis=0), 
    #"LSRTDE" : np.mean(np.abs(data5-goptimum_cec17)//goptimum_cec17, axis=0), 
    #"JADE" : np.mean(np.abs(data6-goptimum_cec17), axis=0)
    })


display(df)
for c in df.columns : print(c, np.mean(df[c]))

print("------------")

df = pd.DataFrame({
    "NLSHADE_RSP" : [MWUT(data1[:, i], data[:,  i]) for i in range(30)], 
    "j2020" : [MWUT(data1[:, i], data2[:,  i]) for i in range(30)], 
    "LSHADE" : [MWUT(data1[:, i], data3[:,  i]) for i in range(30)], 
    #"LSRTDE" : [MWUT(data1[:, i], data5[:,  i]) for i in range(30)], 
    })
for c in df.columns : print(c, np.mean(df[c]))

# CEC 2020

In [ ]:
maxevals=50000
filename = resFolder+"/results_2020_NLSHADE_RSP_5_"+str(maxevals)+".txt"
data = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_ARRDE_5_"+str(maxevals)+".txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_j2020_5_"+str(maxevals)+".txt"
data2 = np.loadtxt(filename, delimiter='\t')
#filename = "./../bin/Release/results_2020_LSHADE_10_"+str(maxevals)+".txt"
#data3 = np.loadtxt(filename, delimiter='\t')
#filename = "./../bin/Release/results_2020_jSO_10_"+str(maxevals)+".txt"
#data4 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_LSRTDE_5_"+str(maxevals)+".txt"
data5 = np.loadtxt(filename, delimiter='\t')

df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec20)/goptimum_cec20, axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec20)/goptimum_cec20, axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec20)/goptimum_cec20, axis=0), 
    #"LSHADE" : np.mean(np.abs(data3-goptimum_cec20), axis=0),
    #"jSO" : np.mean(np.abs(data4-goptimum_cec20), axis=0),
    "LSRTDE" : np.mean(np.abs(data5-goptimum_cec20), axis=0)
    })

display(df)
for c in df.columns : print(c, np.mean(df[c]))

print("------------")

df = pd.DataFrame({
    "NLSHADE_RSP" : [MWUT(data1[:, i], data[:,  i]) for i in range(10)], 
    "j2020" : [MWUT(data1[:, i], data2[:,  i]) for i in range(10)], 
    #"LSHADE" : [MWUT(data1[:, i], data3[:,  i]) for i in range(10)], 
    "LSRTDE" : [MWUT(data1[:, i], data5[:,  i]) for i in range(10)], 
    })
for c in df.columns : print(c, np.mean(df[c]))

In [ ]:
maxevals=1000000
filename = resFolder+"results_2020_NLSHADE_RSP_10_"+str(maxevals)+".txt"
data = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_ARRDE_10_"+str(maxevals)+".txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_j2020_10_"+str(maxevals)+".txt"
data2 = np.loadtxt(filename, delimiter='\t')
#filename = "./../bin/Release/results_2020_LSHADE_10_"+str(maxevals)+".txt"
#data3 = np.loadtxt(filename, delimiter='\t')
#filename = "./../bin/Release/results_2020_jSO_10_"+str(maxevals)+".txt"
#data4 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_LSRTDE_10_"+str(maxevals)+".txt"
data5 = np.loadtxt(filename, delimiter='\t')

df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec20), axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec20), axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec20), axis=0), 
    #"LSHADE" : np.mean(np.abs(data3-goptimum_cec20), axis=0),
    #"jSO" : np.mean(np.abs(data4-goptimum_cec20), axis=0),
    "LSRTDE" : np.mean(np.abs(data5-goptimum_cec20), axis=0)
    })

display(df)
for c in df.columns : print(c, np.mean(df[c]))

print("------------")

df = pd.DataFrame({
    "NLSHADE_RSP" : [MWUT(data1[:, i], data[:,  i]) for i in range(10)], 
    "j2020" : [MWUT(data1[:, i], data2[:,  i]) for i in range(10)], 
    #"LSHADE" : [MWUT(data1[:, i], data3[:,  i]) for i in range(10)], 
    "LSRTDE" : [MWUT(data1[:, i], data5[:,  i]) for i in range(10)], 
    })
for c in df.columns : print(c, np.mean(df[c]))
df


In [ ]:
maxevals=3000000
filename = resFolder+"results_2020_NLSHADE_RSP_15_"+str(maxevals)+".txt"
data = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_ARRDE_15_"+str(maxevals)+".txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_j2020_15_"+str(maxevals)+".txt"
data2 = np.loadtxt(filename, delimiter='\t')
#filename = "./../bin/Release/results_2020_LSHADE_10_"+str(maxevals)+".txt"
#data3 = np.loadtxt(filename, delimiter='\t')
#filename = "./../bin/Release/results_2020_jSO_10_"+str(maxevals)+".txt"
#data4 = np.loadtxt(filename, delimiter='\t')
#filename = resFolder+"results_2020_LSRTDE_10_"+str(maxevals)+".txt"
#data5 = np.loadtxt(filename, delimiter='\t')


df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec20), axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec20), axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec20), axis=0), 
    #"LSHADE" : np.mean(np.abs(data3-goptimum_cec20), axis=0),
    #"jSO" : np.mean(np.abs(data4-goptimum_cec20), axis=0),
 #   "LSRTDE" : np.mean(np.abs(data5-goptimum_cec20), axis=0)
    })

display(df)
for c in df.columns : print(c, np.mean(df[c]))

print("------------")

df = pd.DataFrame({
    "NLSHADE_RSP" : [MWUT(data1[:, i], data[:,  i]) for i in range(10)], 
    "j2020" : [MWUT(data1[:, i], data2[:,  i]) for i in range(10)], 
    #"LSHADE" : [MWUT(data1[:, i], data3[:,  i]) for i in range(10)], 
    #"LSRTDE" : [MWUT(data1[:, i], data5[:,  i]) for i in range(10)], 
    })
for c in df.columns : print(c, np.mean(df[c]))
df

In [ ]:
maxevals=1000000
filename = resFolder+"results_2020_NLSHADE_RSP_20_"+str(maxevals)+".txt"
data = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_ARRDE_20_"+str(maxevals)+".txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_j2020_20_"+str(maxevals)+".txt"
data2 = np.loadtxt(filename, delimiter='\t')
#filename = resFolder+"results_2020_LSRTDE_20_"+str(maxevals)+".txt"
#data3 = np.loadtxt(filename, delimiter='\t')
df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec20), axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec20), axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec20), axis=0), 
    #"LSHADE" : np.mean(np.abs(data3-goptimum_cec20), axis=0),
    #"jSO" : np.mean(np.abs(data4-goptimum_cec20), axis=0),
   # "LSRTDE" : np.mean(np.abs(data5-goptimum_cec20)/goptimum_cec20, axis=0)
    })

display(df)
for c in df.columns : print(c, np.mean(df[c]))

print("------------")

df = pd.DataFrame({
    "NLSHADE_RSP" : [MWUT(data1[:, i], data[:,  i]) for i in range(10)], 
    "j2020" : [MWUT(data1[:, i], data2[:,  i]) for i in range(10)], 
    #"LSHADE" : [MWUT(data1[:, i], data3[:,  i]) for i in range(10)], 
    #"LSRTDE" : [MWUT(data1[:, i], data5[:,  i]) for i in range(10)], 
    })
for c in df.columns : print(c, np.mean(df[c]))
df

# CEC 2022


## 10D

In [ ]:
maxevals=200000
filename = resFolder+"results_2022_NLSHADE_RSP_10_"+str(maxevals)+".txt"
data = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2022_ARRDE_10_"+str(maxevals)+".txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2022_j2020_10_"+str(maxevals)+".txt"
data2 = np.loadtxt(filename, delimiter='\t')
#filename = "./../bin/Release/results_2022_LSHADE_10_"+str(maxevals)+".txt"
#data3 = np.loadtxt(filename, delimiter='\t')
#filename = "./../bin/Release/results_2022_jSO_10_"+str(maxevals)+".txt"
#data4 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2022_LSRTDE_10_"+str(maxevals)+".txt"
data5 = np.loadtxt(filename, delimiter='\t')
df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec22)/goptimum_cec22, axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec22)/goptimum_cec22, axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec22)/goptimum_cec22, axis=0),
    #"LSHADE" : np.mean(np.abs(data3-goptimum_cec22), axis=0),
    #"jSO" : np.mean(np.abs(data4-goptimum_cec22), axis=0),
    "LSRTDE" : np.mean(np.abs(data5-goptimum_cec22)/goptimum_cec22, axis=0),
    })

display(df)
for c in df.columns : print(c, np.mean(df[c]))
print("------------")

df = pd.DataFrame({
    "NLSHADE_RSP" : [MWUT(data1[:, i], data[:,  i]) for i in range(12)], 
    "j2020" : [MWUT(data1[:, i], data2[:,  i]) for i in range(12)], 
    #"LSHADE" : [MWUT(data1[:, i], data3[:,  i]) for i in range(12)], 
    "LSRTDE" : [MWUT(data1[:, i], data5[:,  i]) for i in range(12)], 
    })
for c in df.columns : print(c, np.mean(df[c]))
df

## 20D


In [ ]:
maxevals=1000000
filename = resFolder+"results_2022_NLSHADE_RSP_20_"+str(maxevals)+".txt"
data = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2022_ARRDE_20_"+str(maxevals)+".txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2022_j2020_20_"+str(maxevals)+".txt"
data2 = np.loadtxt(filename, delimiter='\t')
#filename = "./../bin/Release/results_2022_LSHADE_20_"+str(maxevals)+".txt"
#data3 = np.loadtxt(filename, delimiter='\t')
#filename = "./../bin/Release/results_2022_jSO_20_"+str(maxevals)+".txt"
#data4 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2022_LSRTDE_20_"+str(maxevals)+".txt"
data5 = np.loadtxt(filename, delimiter='\t')

df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec22)/goptimum_cec22, axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec22)/goptimum_cec22, axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec22)/goptimum_cec22, axis=0),
    #"LSHADE" : np.mean(np.abs(data3-goptimum_cec22), axis=0),
    #"jSO" : np.mean(np.abs(data4-goptimum_cec22), axis=0),
    "LSRTDE" : np.mean(np.abs(data5-goptimum_cec22)/goptimum_cec22, axis=0),
    })

display(df)
for c in df.columns : print(c, np.mean(df[c]))
print("------------")

df = pd.DataFrame({
    "NLSHADE_RSP" : [MWUT(data1[:, i], data[:,  i]) for i in range(12)], 
    "j2020" : [MWUT(data1[:, i], data2[:,  i]) for i in range(12)], 
    #"LSHADE" : [MWUT(data1[:, i], data3[:,  i]) for i in range(12)], 
    "LSRTDE" : [MWUT(data1[:, i], data5[:,  i]) for i in range(12)], 
    })
for c in df.columns : print(c, np.mean(df[c]))
df